In [1]:
import cv2
import pytesseract
import os
import numpy as np
from ultralytics import YOLO
import joblib

# Expanded list of unsuitable fonts
UNSUITABLE_FONTS = [
    "Comic Sans", "Papyrus", "Brush Script", "Chalkduster", "Impact", "Stencil", 
    "Courier", "Lucida Handwriting", "Jokerman", "Edwardian Script", "Kristen ITC", 
    "Harlow Solid", "Snap ITC", "Curlz MT", "Vivaldi", "Giddyup Std", "Bradley Hand", 
    "Old English Text", "Wide Latin", "Showcard Gothic", "Playbill", "Cooper Black", 
    "Algerian", "Pristina", "Magneto", "Viner Hand ITC", "Mistral", "Forte", 
    "Ravie", "Jester", "Kunstler Script", "Onyx", "Elephant", "Blackadder ITC", 
    "Monotype Corsiva", "Bodoni MT Poster", "Gill Sans Ultra Bold", "Copperplate Gothic Bold", 
    "Copperplate Gothic Light", "Broadway", "Perpetua Titling MT", "Rockwell Extra Bold", 
    "Berlin Sans FB", "Engravers MT", "Castellar", "Zapfino", "Segoe Script", 
    "Lucida Calligraphy", "Freestyle Script", "Stencil Std", "Garamond Bold", 
    "Baskerville Old Face", "Bookman Old Style"
]

ACCESSIBLE_FONT_SIZE = 14  # Minimum size in points for accessibility

# Load YOLO model for text detection
def load_yolo_model():
    return YOLO("yolov8n.pt")

model = load_yolo_model()

# Function to calculate font size from bounding box height (approximate)
def calculate_font_size(bounding_box_height):
    scale_factor = 0.75  # Adjust scale factor to map pixel height to points
    return bounding_box_height * scale_factor

# Function to evaluate font size compliance (14pt or above is accessible)
def is_accessible_font_size(detected_font_size):
    return detected_font_size >= ACCESSIBLE_FONT_SIZE

# Function to evaluate font style compliance
def is_accessible_font_style(text_image):
    ocr_result = pytesseract.image_to_string(text_image, config='--psm 6')
    # Check if any unsuitable font is detected
    return not any(font in ocr_result for font in UNSUITABLE_FONTS)

# Process a single poster for font size and style compliance
def process_poster(poster_path, model):
    print(f"Processing poster: {poster_path}")
    image = cv2.imread(poster_path)
    if image is None:
        print("Failed to load image.")
        return 0, 0

    results = model(poster_path)
    font_size_accessible_count = 0
    font_style_accessible_count = 0
    text_regions_count = 0

    for result in results:
        boxes = result[:, :4]  # Extract x1, y1, x2, y2 from the tensor

        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            text_region = image[y1:y2, x1:x2]  # Crop text region

            # Calculate and evaluate font size
            detected_font_size = calculate_font_size(y2 - y1)
            if is_accessible_font_size(detected_font_size):
                font_size_accessible_count += 1

            # Evaluate font style
            if is_accessible_font_style(text_region):
                font_style_accessible_count += 1

            text_regions_count += 1

    # Determine majority for font size and style
    font_size_score = 100 if font_size_accessible_count > text_regions_count / 2 else 0
    font_style_score = 100 if font_style_accessible_count > text_regions_count / 2 else 0

    print(f"Font Size Score: {font_size_score}, Font Style Score: {font_style_score}")
    return font_size_score, font_style_score

# Save the model and process function
def save_joblib_model():
    joblib.dump((model, process_poster), "poster_evaluation_model.joblib")
    print("Model saved as poster_evaluation_model.joblib")

# Load the model and process function
def load_joblib_model():
    loaded_model, loaded_process_function = joblib.load("poster_evaluation_model.joblib")
    return loaded_model, loaded_process_function

# Function to evaluate a single uploaded poster
def evaluate_uploaded_poster(poster_path):
    loaded_model, loaded_process_function = load_joblib_model()
    font_size_score, font_style_score = loaded_process_function(poster_path, loaded_model)
    print(f"Poster: {poster_path}, Font Size Score: {font_size_score}%, Font Style Score: {font_style_score}%")

# Save the model for the first time
save_joblib_model()

c:\Users\susha\Desktop\FontSizeandStyleEvaluation\myenv\lib\site-packages\ultralytics\nn\tasks.py:341: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(attemp

Model saved as poster_evaluation_model.joblib


In [6]:
evaluate_uploaded_poster("C:\\Users\\susha\\Desktop\\FontSizeandStyleEvaluation\\Posters\\1.png")

Processing poster: C:\Users\susha\Desktop\FontSizeandStyleEvaluation\Posters\1.png


Ultralytics YOLOv8.0.0  Python-3.10.5 torch-2.5.0+cpu CPU
Fusing layers... 
YOLOv8n summary: 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs
image 1/1 C:\Users\susha\Desktop\FontSizeandStyleEvaluation\Posters\1.png: 448x640 1 book, 211.8ms
Speed: 0.0ms pre-process, 211.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


Font Size Score: 100, Font Style Score: 100
Poster: C:\Users\susha\Desktop\FontSizeandStyleEvaluation\Posters\1.png, Font Size Score: 100%, Font Style Score: 100%
